# Cargando maestro dataset preprocesado

In [13]:
!find "./maestro_encoded" -name "*.txt" > maestro_encoded_paths.txt

In [16]:
with open("maestro_encoded_paths.txt") as f:
    paths = list(map(lambda x: x.strip(), f.readlines()))

In [18]:
def read_file(path):
    with open(path) as f:
        tokens = f.readline().split(',')
    return tokens

In [19]:
read_file(paths[0])

['TIMESHIFT<46>',
 'NOTE-ON<54>',
 'SET-VELOCITY<84>',
 'NOTE-ON<61>',
 'NOTE-ON<64>',
 'NOTE-ON<69>',
 'NOTE-ON<73>',
 'NOTE-ON<76>',
 'NOTE-ON<81>',
 'TIMESHIFT<1>',
 'NOTE-OFF<64>',
 'NOTE-OFF<69>',
 'TIMESHIFT<1>',
 'NOTE-OFF<54>',
 'NOTE-OFF<61>',
 'NOTE-OFF<81>',
 'TIMESHIFT<1>',
 'NOTE-OFF<73>',
 'NOTE-OFF<76>',
 'TIMESHIFT<49>',
 'NOTE-ON<30>',
 'SET-VELOCITY<87>',
 'NOTE-ON<57>',
 'TIMESHIFT<1>',
 'NOTE-ON<49>',
 'SET-VELOCITY<76>',
 'NOTE-ON<52>',
 'TIMESHIFT<5>',
 'NOTE-OFF<30>',
 'SET-VELOCITY<31>',
 'NOTE-ON<33>',
 'TIMESHIFT<3>',
 'NOTE-OFF<33>',
 'TIMESHIFT<1>',
 'NOTE-ON<30>',
 'SET-VELOCITY<65>',
 'TIMESHIFT<3>',
 'NOTE-OFF<30>',
 'TIMESHIFT<2>',
 'NOTE-ON<33>',
 'SET-VELOCITY<57>',
 'TIMESHIFT<1>',
 'NOTE-OFF<33>',
 'TIMESHIFT<2>',
 'NOTE-ON<30>',
 'SET-VELOCITY<63>',
 'TIMESHIFT<1>',
 'NOTE-OFF<30>',
 'TIMESHIFT<3>',
 'NOTE-ON<33>',
 'SET-VELOCITY<60>',
 'TIMESHIFT<1>',
 'NOTE-OFF<33>',
 'TIMESHIFT<2>',
 'NOTE-ON<30>',
 'SET-VELOCITY<68>',
 'TIMESHIFT<1>',
 'NOTE-OFF

In [20]:
vocabulary = read_file("vocabulary.txt")
vocabulary

['TIMESHIFT<855>',
 'NOTE-ON<48>',
 'TIMESHIFT<761>',
 'TIMESHIFT<1022>',
 'NOTE-ON<71>',
 'SET-VELOCITY<110>',
 'TIMESHIFT<1021>',
 'NOTE-ON<36>',
 'TIMESHIFT<53>',
 'TIMESHIFT<45>',
 'NOTE-OFF<60>',
 'TIMESHIFT<143>',
 'TIMESHIFT<789>',
 'TIMESHIFT<447>',
 'TIMESHIFT<412>',
 'TIMESHIFT<594>',
 'SET-VELOCITY<106>',
 'TIMESHIFT<275>',
 'NOTE-ON<49>',
 'TIMESHIFT<218>',
 'TIMESHIFT<6>',
 'TIMESHIFT<422>',
 'SET-VELOCITY<44>',
 'TIMESHIFT<294>',
 'TIMESHIFT<308>',
 'TIMESHIFT<435>',
 'SET-VELOCITY<34>',
 'NOTE-OFF<33>',
 'TIMESHIFT<1094>',
 'SET-VELOCITY<119>',
 'TIMESHIFT<338>',
 'TIMESHIFT<327>',
 'TIMESHIFT<1268>',
 'TIMESHIFT<314>',
 'TIMESHIFT<224>',
 'TIMESHIFT<198>',
 'TIMESHIFT<479>',
 'TIMESHIFT<191>',
 'TIMESHIFT<66>',
 'SET-VELOCITY<107>',
 'TIMESHIFT<162>',
 'TIMESHIFT<267>',
 'TIMESHIFT<264>',
 'TIMESHIFT<313>',
 'TIMESHIFT<280>',
 'NOTE-OFF<22>',
 'TIMESHIFT<10>',
 'TIMESHIFT<414>',
 'NOTE-OFF<45>',
 'TIMESHIFT<734>',
 'TIMESHIFT<318>',
 'TIMESHIFT<130>',
 'NOTE-ON<67>',
 '

In [21]:
len(vocabulary)  # Este tamaño se disminuira discretizando el 'velocity'

912

# Usando gpt-2 de huggingface

In [22]:
from __future__ import print_function

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available(), device

(True, device(type='cuda'))

In [27]:
class GPT2MusicTokensWrapper:
    
    def __init__(self):
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.model.to(device)

    @staticmethod
    def adjust_length_to_model(length, max_sequence_length):
        if length < 0 and max_sequence_length > 0:
            length = max_sequence_length
        elif 0 < max_sequence_length < length:
            length = max_sequence_length  # No generation bigger than model size
        elif length < 0:
            length = MAX_LENGTH  # avoid infinite loop
        return length

    def continue_text(self, encoded_text, text_to_complete, length=20):
        
        encoded_text = encoded_text.to(device)

        max_length = self.model.config.max_position_embeddings
        length = self.adjust_length_to_model(length, max_sequence_length=max_length)

        output_sequences = self.model.generate(
            input_ids=encoded_text,
            max_length=length + len(encoded_text[0]),
            temperature=1.0,
            top_k=0,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=1.0,
            do_sample=True,
        )

        return self.tokenizer.decode(output_sequences.tolist()[0], clean_up_tokenization_spaces=True)


In [28]:
wrapper = GPT2MusicTokensWrapper()

In [29]:
wrapper.model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye